### Auxiliar Functions

In [9]:
def print_red(text):
    print('\x1b[31m' + text + '\x1b[0m')
    
def print_yellow(text):
    print('\x1b[33m' + text + '\x1b[0m')

def print_green(text):
    print('\x1b[32m' + text + '\x1b[0m')

def print_pink(text):
    print('\x1b[35m' + text + '\x1b[0m')

def print_cyan(text):
    print('\x1b[36m' + text + '\x1b[0m')

## Get the Instances / Ingredient list
The instances have been generated using the notebook: Generate Instance.ipynb

In [10]:
import os
import re

instances_folder = '..\instances'
instances_path = [os.path.join(instances_folder, f) for f in os.listdir(instances_folder) if f.endswith('.dzn')]
# Sort
def numeric_sort_key(file_path):
    match = re.search(r'\d+', file_path)
    return int(match.group()) if match else file_path

instances_path = sorted(instances_path, key=numeric_sort_key)
# print(instances_path)

# Minizinc
### Chosen model with annotations
The models have been generated using the notebook: GenerateAnnotations.ipynb

In [11]:
model = '../models/largest-indomain_max.mzn'

### Solvers

In [12]:
!minizinc --solvers
solvers = ["HiGHS", "COIN-BC"]

MiniZinc driver.
Available solver configurations:
  Chuffed 0.11.0 (org.chuffed.chuffed, cp, lcg, int)
  COIN-BC 2.10.8/1.17.7 (org.minizinc.mip.coin-bc, mip, float, api, osicbc, coinbc, cbc)
  CPLEX <unknown version> (org.minizinc.mip.cplex, mip, float, api)
  findMUS 0.7.0 (org.minizinc.findmus)
  Gecode 6.3.0 (org.gecode.gecode, default solver, cp, int, float, set, restart)
  Gecode Gist 6.3.0 (org.gecode.gist, cp, int, float, set, restart)
  Globalizer 0.1.7.2 (org.minizinc.globalizer, experimental, tool)
  Gurobi <unknown version> (org.minizinc.mip.gurobi, mip, float, api)
  HiGHS 1.4.2 (org.minizinc.mip.highs, mip, float, api, highs)
  OR-TOOLS 9.6.0 (com.google.or-tools)
  SCIP <unknown version> (org.minizinc.mip.scip, mip, float, api)
  Xpress <unknown version> (org.minizinc.mip.xpress, mip, float, api)
Search path for solver configurations:
  C:\Users\juanj\AppData\Roaming/MiniZinc/solvers
  C:\Program Files\MiniZinc\share\minizinc/solvers


## Calculate labels

In [13]:
import random
import subprocess
# Returns the solver who solved the model in the best way possible
def solve_with_minizinc(instances, timeout_mzn, timeout):    
    winner = [0] * len(instances) # 0 for HiGHS, 1 for COIN-BC
    times = []
    timeout_mzn = timeout_mzn*1000
    
    for i, instance in enumerate(instances):
        mnt = float('inf')
        time = float('inf')
        times.append([])
        for j, solver in enumerate(solvers):
            # Run the minizinc command for the current model and data file  
            cmd = f"minizinc --solver {solver} --output-time {model} {instance} --output-time --solver-time-limit {timeout_mzn}" 
            proc = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE)
            try: 
                # Get output from fzn
                stdout, stderr = proc.communicate(timeout=timeout) 
                stdout = stdout.decode()
                try:
                    # Store value mnt (temp).
                    start = stdout.find('mnt = ')+6
                    end = stdout.find(';',start)
                    mnt_temp = int(stdout[start:end])
                    # Store value time (temp).
                    time_temp = float(stdout.split(' ')[-2])
                except:
                    mnt_temp = float('inf')
                    time_temp = float('inf')
                # print_green(f'{solver} with {instance} SOLVED with a value of {mnt_temp} in {time_temp:.2f} seconds.')
                
            except subprocess.TimeoutExpired: 
                print_red(f'Error from minizinc: Stopping.')                
                mnt_temp = float('inf')
                time_temp = float('inf')
                if os.name == 'nt': # If the os is Windows
                    subprocess.call(['taskkill', '/F', '/T', '/PID', str(proc.pid)]) # Force kill the process
                else:
                    os.killpg(os.getpgid(proc.pid), signal.SIGTERM) # Kill the process group
            
            # Store values for futures generations
            times[i].append(time_temp)
            
            if mnt >= mnt_temp: # If the value is minor from the stored
                if mnt == mnt_temp: # If the value is equal from the stored compare time
                    if time >= time_temp: # If the value of time from minizinc is minor from the stored
                        if time == time_temp: # If the value of time from minizinc is equal from the stored choose random
                            winner[i] = random.randint(j-1,j)    
                        else:
                            mnt = mnt_temp
                            time = time_temp
                            winner[i] = j
                else:
                    mnt = mnt_temp
                    time = time_temp
                    winner[i] = j
        print_yellow(str(i)+ ": " + solvers[winner[i]]+" WON!")
    return winner, times

In [14]:
labels, times = solve_with_minizinc(instances_path, 10, 15)

0: COIN-BC WON!
1: COIN-BC WON!
2: HiGHS WON!
3: HiGHS WON!
4: COIN-BC WON!
5: HiGHS WON!
6: COIN-BC WON!
7: COIN-BC WON!
8: COIN-BC WON!
9: HiGHS WON!
10: HiGHS WON!
11: COIN-BC WON!
12: COIN-BC WON!
13: HiGHS WON!
14: HiGHS WON!
15: COIN-BC WON!
16: COIN-BC WON!
17: COIN-BC WON!
18: HiGHS WON!
19: COIN-BC WON!
20: HiGHS WON!
Error from minizinc: Stopping.
21: COIN-BC WON!
22: COIN-BC WON!
23: COIN-BC WON!
24: COIN-BC WON!
25: COIN-BC WON!
26: COIN-BC WON!
27: HiGHS WON!
28: COIN-BC WON!
29: HiGHS WON!
30: COIN-BC WON!
31: COIN-BC WON!
32: COIN-BC WON!
33: COIN-BC WON!
34: HiGHS WON!
35: HiGHS WON!
36: COIN-BC WON!
37: COIN-BC WON!
38: COIN-BC WON!
39: HiGHS WON!
40: COIN-BC WON!
41: COIN-BC WON!
42: HiGHS WON!
43: HiGHS WON!
44: COIN-BC WON!
45: COIN-BC WON!
46: HiGHS WON!
47: COIN-BC WON!
48: COIN-BC WON!
49: HiGHS WON!
50: COIN-BC WON!
51: HiGHS WON!
52: HiGHS WON!
53: HiGHS WON!
54: COIN-BC WON!
55: COIN-BC WON!
56: HiGHS WON!
57: COIN-BC WON!
58: COIN-BC WON!
59: COIN-BC WON!
60:

## Storing labels and times

In [15]:
path = '../instances/labels.txt'
with open(path, 'w', encoding='utf8') as file_object:
    file_object.write(str(labels))
    
path = '../instances/times.txt'
with open(path, 'w', encoding='utf8') as file_object:
    file_object.write('solvers:\n'+str(solvers)+'\nTimes that took each solver with a timeout of 10s for each instance.\n(Values greater than 10s was because minizinc was looking for a last solution).\n'+str(times))

In [16]:
freq = {}

# Count frequency of each element in the array
for num in labels:
    if num in freq:
        freq[num] += 1
    else:
        freq[num] = 1

# Print frequency of each element
for num, count in freq.items():
    print(f"{num}: {count} {count/len(labels)*100}%")

1: 183 61.0%
0: 117 39.0%
